In [2]:
import warnings
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape
from keras.layers.merge import concatenate, multiply,add
import tensorflow as tf
from keras import regularizers
from keras.initializers import glorot_uniform
from tqdm import tqdm
from keras import regularizers
from keras.models import load_model
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

Using TensorFlow backend.


# Reading Datasets

In [25]:
# Main Dataset
data= pd.read_csv("Data/cif_dataset_complete.csv",header=None)
# Expert Knowledge
predictions = pd.read_csv("Data/theta_25_horg.csv",index_col=0,skiprows = [1])
print(data.shape)
print(predictions.shape)

(72, 123)
(72, 90)


# Functions to Make Inputs

In [6]:
def make_input(data,window_size,horizon=1):
    length=data.shape[0]
#     depth=data.shape[2]
    y = np.zeros([length-window_size+1-horizon,horizon])
    output=np.zeros([length-window_size+1-horizon,window_size])
    for i in range(length-window_size+1-horizon):
        output[i:i+1,:]=data[i:i+window_size]
        y[i,:]= data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def make_k_input(data,window_size,horizon):
    length = data.shape[0]
    output= np.zeros([length-window_size+1-horizon,horizon])
    for i in range(length-window_size-horizon+1):
        output[i:i+1,:]=data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_input(data,window_size,horizon=1):
    length=data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
#     print(str(extra))
    data = np.append(data,np.zeros([horizon-extra]))
#     print(data)
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
        
    output=np.zeros([i_val,window_size])
    y=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[i*horizon:(i*horizon)+window_size]
        y[i,:]= data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
        
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_k_input(data,window_size,horizon):
    length = data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
    data_app = np.repeat(data[-1],extra)
    data = np.append(data,data_app)    
#     data = np.append(data,np.zeros([horizon-extra]))
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
    output=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)    

# Evaluation Function (SMAPE)

In [15]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred))
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 200.0 * np.mean(diff)

# Auto-Encoder for Expert Knowledge

In [59]:
theta_length = predictions.shape[0]

horizon=6
window_size=6
theta_all=np.zeros(0)
for i in range(theta_length):
    current_pred= np.asarray(predictions.iloc[i].dropna().values,dtype=float)
    theta_all=np.concatenate((theta_all,current_pred),axis=0)
theta_input = make_k_input(theta_all,window_size,horizon)
normalized_theta_input=np.zeros(theta_input.shape)
for j in range(theta_input.shape[0]):
    scaler = MinMaxScaler()
    scaler.fit(theta_input[j])
    normalized_theta_input[j]= scaler.transform(theta_input[j])
#######################################################################################
norm_reshaped_input=normalized_theta_input.reshape(normalized_theta_input.shape[0],6)
ncol = normalized_theta_input.shape[1] #6

encoding_dim = 4
input_dim = Input(shape = (ncol, ))
# Encoder Layers
encoded1 = Dense(5, activation = 'relu')(input_dim)
encoded2 = Dense(encoding_dim, activation = 'relu')(encoded1)
# Decoder Layers
decoded1 = Dense(5, activation = 'relu')(encoded2)
decoded2 = Dense(ncol, activation = 'sigmoid')(decoded1)
# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded2)
autoencoder.compile(optimizer = 'adam', loss = 'mse')

autoencoder.fit(norm_reshaped_input, norm_reshaped_input, epochs = 100, batch_size = 32, shuffle = False,verbose=2)
encoder = Model(inputs = input_dim, outputs = encoded2)
encoded_input = Input(shape = (encoding_dim, ))
auto_out = np.array(encoder.predict(norm_reshaped_input))

new_theta_input=auto_out.reshape(auto_out.shape[0],auto_out.shape[1],1)
autoencoder.save('Output/autoencoder_model.h5')
encoder.save('Output/encoder_model.h5')
np.savetxt('Output/new_theta_input.csv',auto_out, fmt='%1.3f',delimiter=',')    

Epoch 1/100
 - 1s - loss: 0.1338
Epoch 2/100
 - 0s - loss: 0.0897
Epoch 3/100
 - 0s - loss: 0.0708
Epoch 4/100
 - 0s - loss: 0.0588
Epoch 5/100
 - 0s - loss: 0.0494
Epoch 6/100
 - 0s - loss: 0.0416
Epoch 7/100
 - 0s - loss: 0.0353
Epoch 8/100
 - 0s - loss: 0.0301
Epoch 9/100
 - 0s - loss: 0.0247
Epoch 10/100
 - 0s - loss: 0.0205
Epoch 11/100
 - 0s - loss: 0.0170
Epoch 12/100
 - 0s - loss: 0.0151
Epoch 13/100
 - 0s - loss: 0.0142
Epoch 14/100
 - 0s - loss: 0.0134
Epoch 15/100
 - 0s - loss: 0.0127
Epoch 16/100
 - 0s - loss: 0.0124
Epoch 17/100
 - 0s - loss: 0.0121
Epoch 18/100
 - 0s - loss: 0.0118
Epoch 19/100
 - 0s - loss: 0.0116
Epoch 20/100
 - 0s - loss: 0.0115
Epoch 21/100
 - 0s - loss: 0.0113
Epoch 22/100
 - 0s - loss: 0.0112
Epoch 23/100
 - 0s - loss: 0.0111
Epoch 24/100
 - 0s - loss: 0.0110
Epoch 25/100
 - 0s - loss: 0.0109
Epoch 26/100
 - 0s - loss: 0.0108
Epoch 27/100
 - 0s - loss: 0.0107
Epoch 28/100
 - 0s - loss: 0.0106
Epoch 29/100
 - 0s - loss: 0.0104
Epoch 30/100
 - 0s - lo

# Case 1 : Only Data

In [56]:
warnings.filterwarnings('ignore')
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    final_smape=np.zeros(data_length)
    for y in range(data_length):
        num_test = data.iloc[y].values[1]
        horizon=6
        window_size=8
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
        series_data = series_d[:-num_test]
        series_length = series_data.size
        current_test=series_d[-num_test:]
        n_val = int(np.round(series_length*.2))
        if n_val < horizon: 
            n_val = horizon            
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        

        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)
        
        scaler = MinMaxScaler()
        scaler.fit(series_data.reshape(series_data.shape[0],1))
        for j in range(train_sequence[0].shape[0]):
            train_sequence_norm[0][j]= scaler.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])    
        
        for j in range(val_sequence[0].shape[0]):
            val_sequence_norm[0][j]= scaler.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            test_sequence_norm[0][j]= scaler.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])

        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
        
        train_input = x_train
        val_input = x_val
        test_input = x_test
####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        tf.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        
        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.01))(branch_3)        

        model=Model(inputs=[input_data],outputs=net)

        callback = ModelCheckpoint(filepath='Output/checkpoint_CIF_01.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))
        model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=100,verbose=0)

        model.load_weights('Output/checkpoint_CIF_01.h5')
        pred=model.predict({'input_data':test_input})
        pred=scaler.inverse_transform(pred)

        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test)
        final_smape[y]=AAA
        pbar.update(1)

np.savetxt('Output/prediction_CIF_01.csv',final_predictions, fmt='%1.3f',delimiter=',')
model.summary()
print('-----------------')
print("SMAPE:")
print (sum(final_smape)/data_length)    

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [03:57<00:00,  2.07s/it]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_data (InputLayer)      (None, 8, 1)              0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 8, 32)             128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 8, 32)             3104      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 8, 32)             3104      
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_final (Dense)          (None, 6)                 1542      
Total params: 7,878
Trainable params: 7,878
Non-trainable params: 0
_________________________________________________________________
--------

# Case 2: Data + Expert Knowledge

In [55]:
warnings.filterwarnings('ignore')
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    final_smape=np.zeros(data_length)

    for y in range(data_length):
        num_test = data.iloc[y].values[1]
        horizon=6
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] 
        series_data = series_d[:-num_test]
        series_length = series_data.size
        current_test=series_d[-num_test:]
        n_val = int(np.round(series_length*.2))        
        if n_val < horizon: 
            n_val = horizon                                
        LL=series_data.shape[0]-n_val
        HH=LL-window_size+1-horizon
        if (HH<0):
            train = series_data[:-n_val+1-HH]
        else:
            train = series_data[:-n_val]                        

        test = series_d[-(num_test+window_size):]
        val = series_data[-(n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)
        
        scaler = MinMaxScaler()
        scaler.fit(series_data.reshape(series_data.shape[0],1))
        for j in range(train_sequence[0].shape[0]):
            train_sequence_norm[0][j]= scaler.transform(train_sequence[0][j])
            train_sequence_norm[1][j]= scaler.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])    
        for j in range(val_sequence[0].shape[0]):
            val_sequence_norm[0][j]= scaler.transform(val_sequence[0][j])
            val_sequence_norm[1][j]= scaler.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            test_sequence_norm[0][j]= scaler.transform(test_sequence[0][j])
            test_sequence_norm[1][j]= scaler.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])            

        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]        
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
        
        train_input = x_train
        val_input = x_val
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Expert Knowledge####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
        series_p=current_pred
        series_pred=series_p[:-num_test]        
        if (HH<0):
            train_p = series_pred[:-n_val+1-HH]
        else:
            train_p = series_pred[:-n_val]                                        
        val_p = series_pred[-(n_val+window_size):]
        test_p = series_p[-(num_test+window_size):]

        train_pred = make_k_input(train_p,window_size,horizon)
        val_pred = make_k_input(val_p,window_size,horizon)
        test_pred = nonov_make_k_input(test_p,window_size,horizon)        
                
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
        
        scaler_pred = MinMaxScaler()
        scaler_pred.fit(series_pred.reshape(series_pred.shape[0],1))
        for j in range(train_pred.shape[0]):
            train_pred_norm[j]= scaler_pred.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            val_pred_norm[j]= scaler_pred.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            test_pred_norm[j]= scaler_pred.transform(test_pred[j])
        
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],train_pred_norm.shape[1])
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],val_pred_norm.shape[1])
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],test_pred_norm.shape[1])
##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('Output/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))
                
        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)                    
####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        tf.reset_default_graph()
        K.clear_session()
        
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,encoding_dim),name='input_pred')
        
        encoded12 = Dense(8, activation = 'relu')(input_pred)
        encoded22 = Dense(16, activation = 'relu')(encoded12)
        encoded32 = Dense(32, activation = 'relu')(encoded22)
        encoded32=Reshape((1,32))(encoded32)
        
        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=concatenate([branch_0,encoded32],axis=1)
        branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.01))(branch_3)
        
        model=Model(inputs=[input_data,input_pred],outputs=net)
        callback = ModelCheckpoint(filepath='Output/checkpoint_CIF_02.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))
        model.fit({'input_data':train_input,'input_pred':train_pred_norm_auto},y_train,validation_data=[[val_input,val_pred_norm_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=200,verbose=0)
        model.load_weights('Output/checkpoint_CIF_02.h5')
        
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_auto})
        pred=scaler.inverse_transform(pred)
        
        final_predictions[y,:num_test] = pred.reshape(num_test)
        final_test[y,:num_test]=test[-num_test:]

        AAA=smape(pred.reshape(num_test),current_test)
        final_smape[y]=AAA
        pbar.update(1)

np.savetxt('Output/prediction_CIF_02.csv',final_predictions, fmt='%1.3f',delimiter=',')
model.summary()
print('-----------------')
print("SMAPE:")
print (sum(final_smape)/data_length)

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [09:33<00:00,  4.63s/it]


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pred (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 8)            40          input_pred[0][0]                 
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 16)           144         dense_1[0][0]                    
__________________________________________________________________________________________________
input_data (InputLayer)         (None, 6, 1)         0                                            
__________________________________________________________________________________________________
dense_3 (D